### PIL (Pillow) is for oipen an image
###### this library can read and convert image to binary file for processing in python
### OpenCV is for change an image
###### this library is for change an imageis for prossessing on image to prepare it for ocr like resize and etc
### Tessereact (Pytesseract) is for OCR an Image
###### this is an engine for ocr

In [1]:
import cv2
import PIL
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Pilllow

### loading images

In [ ]:
im_file = r'C://Users/Omid roodi/Desktop/OCR_test/tutorial/import_1.jpg'

### Image.open() is for openning image in memory and when you print it you can see important details about image
### for choose specific detail from image that opened you can for example print(im.size)

In [ ]:
im = Image.open(im_file)
print(im.size)

### for show the imported image you can use im.show()

In [ ]:
im.show()

### you can rotate image: im.rotate(degree).show()
### you can crop image: 

In [ ]:
im.rotate(90).show()

### saving file

In [ ]:
im.save("C://Users/Omid roodi/Desktop/OCR_test/tutorial/import_2.jpg")

# Open-CV

## preprocessing images for OCR

### Open image

In [ ]:
image_file = im_file

In [ ]:
img= cv2.imread(image_file)

In [ ]:
cv2.imshow('test',img)
cv2.waitKey(0)

In [ ]:
def display_im(im_path):
    dpi = 80
    im_data = plt.imread(im_path)

    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [ ]:
display_im(image_file)

### inverted images

#### make all light areas to dark and dark areas to light
#### in tesseract downdate versions this step is good but in latest version it can cuse poor results

In [ ]:
inverted_image= cv2.bitwise_not(img)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/inverted.jpg",inverted_image)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/inverted.jpg")

### Rescaling

#### there is optimal size of image for getting good OCR result and this is defined by the height of characters which comes down to dpi

### Binarization

#### this method is for converting image into black and white

In [ ]:
def grayscale(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [ ]:
gray_image= grayscale(img)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/grayscal.jpg",gray_image)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/grayscal.jpg")

In [ ]:
thresh , im_bw = cv2.threshold(gray_image, 200, 280, cv2.THRESH_BINARY)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/im_bw.jpg",im_bw)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/im_bw.jpg")

## Noise Removal

#### remove pixels that not correspond to main text

#### befor do this step test OCR and when it goes wrong do this step

#### the morphology and medianblur is stands for noise removal

In [ ]:
def noise_removal (image):
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1),  np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return image

In [ ]:
no_noise = noise_removal(im_bw)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/no_noise.jpg",no_noise)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/no_noise.jpg")

## Dilation and erosion

#### its for thic and thin letters
#### if you dont whant noise removal you can do this step because in noise removal step we do this too

In [ ]:
def thin_font (image):
    image= cv2.bitwise_not(image)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

In [ ]:
eroded_image= thin_font(im_bw)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/eroded_image.jpg",eroded_image)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/eroded_image.jpg")

In [ ]:
def thik_font (image):
    image= cv2.bitwise_not(image)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return image

In [ ]:
dilated_image= thin_font(eroded_image)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/dilated_image.jpg",dilated_image)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/dilated_image.jpg")

## Rotation or Deskewing

####  is for rotate or not enogh is stright or deskesed way

In [ ]:
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
    cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

In [ ]:
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [ ]:
fixed = deskew(img)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/fixed.jpg",fixed)

In [ ]:
display_im("C://Users/Omid roodi/Desktop/OCR_test/tutorial/fixed.jpg")

## Remove or fill missing borders

In [ ]:
def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

In [ ]:
no_borders = remove_borders(no_noise)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/no_borders.jpg", no_borders)
display("C://Users/Omid roodi/Desktop/OCR_test/tutorial/no_borders.jpg")

In [ ]:
color = [255, 255, 255]
top, bottom, left, right = [150]*4

In [ ]:
image_with_border = cv2.copyMakeBorder(no_borders, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
cv2.imwrite("C://Users/Omid roodi/Desktop/OCR_test/tutorial/image_with_border.jpg", image_with_border)
display("C://Users/Omid roodi/Desktop/OCR_test/tutorial/image_with_border.jpg")

## transparenty

In [ ]:
def convertImage():
    img = Image.open("C://Users/Omid roodi/Desktop/OCR_test/tutorial/no_noise.jpg")
    img = img.convert("RGBA")
 
    datas = img.getdata()
 
    newData = []
 
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
 
    img.putdata(newData)
    img.save("C://Users/Omid roodi/Desktop/OCR_test/tutorial/transparent_image.png")
    print("Successful")

In [ ]:
convertImage()

# Pytesseract

In [40]:
im_file = r'C://Users/Omid roodi/Desktop/OCR_test/tutorial/test_blur.jpg'

In [41]:
img = cv2.imread(im_file)

In [52]:
result = pytesseract.image_to_string(img, lang='fas',config='--psm 6 --oem 3')

In [53]:
print(result)

۳(
وا فا
۹۷۳۹
۱۷۳۰۰۰
۱۸-۰۳۷
۴۴۵,۰۶
۰("
(۶۳۸,۹۲۱,۹۶۰)
(۸:۳۱۳,۳۴۱)
(۵۲,۶۳۲)
(۱۸,۰۷۷,۵۷۶)
(۴۰۸,۹۳۴,۹۹۸)
یش
(4۹۹,۸۰۹,۱۰۴)
(۱۳۱,۷۱۶,۷۵۲)
(۴۵,۶۲۳,۹۵۱)
(۱۱۱,۰۷۶,۹۷۸)
۳



In [58]:
import percato
import hezar
from hezar.models import Model

c:\Users\Omid roodi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
# model = Model.load("hezarai/trocr-base-fa-v2")
# texts = model.predict([img])
# print(f"TrOCR Output: {texts}")

# OCR with CRNN
model = Model.load("hezarai/crnn-fa-printed-96-long")
texts = model.predict(img)
print(f"CRNN Output: {texts}")

model_config.yaml: 100%|██████████| 871/871 [00:00<00:00, 218kB/s]


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.huggingface.co\', port=443): Max retries exceeded with url: /repos/1f/9f/1f9f0404ecff505753a4dc60abc5e9d9babfd6aa07e1bfb974c31a7ff0e3e5ea/232a6234b84737f705fc39fa74d31d6ff52c44ec5277f78bd782924e544a5284?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.pt%3B+filename%3D%22model.pt%22%3B&Expires=1701516844&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMTUxNjg0NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzFmLzlmLzFmOWYwNDA0ZWNmZjUwNTc1M2E0ZGM2MGFiYzVlOWQ5YmFiZmQ2YWEwN2UxYmZiOTc0YzMxYTdmZjBlM2U1ZWEvMjMyYTYyMzRiODQ3MzdmNzA1ZmMzOWZhNzRkMzFkNmZmNTJjNDRlYzUyNzdmNzhiZDc4MjkyNGU1NDRhNTI4ND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=JhpSfmMR3lAoTDvnEb33Be1jiMUAwYukSWdUksDpDOruHIvGp~jf-esgD5~AEQXEx01t6JdEVD-SXiOo6qsCJrxqx-Vdgxmnfq7FrAadedzpNYEs4TiGHuq3IVWuTMkb0MTFfGof72SJSkhfv9W1OMrsmQ~XtxHKagldho8KH-gjz4682umkmkfy